<center><div class="alert alert-block alert-info" style=" line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 18px; color:green"> &nbsp; Bengali.AI Speech Recognition : Our very first submission</b><br><br><b style="font-size: 18px; color:green">HELLO EVERYONE!</b><br>
</div></center>

In this Notebook we'll try to submit our first submission by using a publicly available model.

We'll be using this [BanglaASR](https://huggingface.co/bangla-speech-processing/BanglaASR) model. It is actually a finetuned [whisper](https://openai.com/research/whisper) model on rained [Bangla Mozilla Common Voice Dataset](https://arxiv.org/abs/2206.14053). It's reported performance is 4.58% on 7k validation set which seems pretty good to be a starting point. So let's start with that.

I'll be trying to use the other publicly available models one by one and see how they perform. Then we'll get an idea from where we can start the training/finetuning on this huge dataset.

In [2]:
print("hello")

hello


# Imports

In [2]:
import os
import librosa
import torch
import torchaudio
import numpy as np

from transformers import WhisperTokenizer
from transformers import WhisperProcessor
from transformers import WhisperFeatureExtractor
from transformers import WhisperForConditionalGeneration
from tqdm import tqdm
import pandas as pd
import soundfile as sf
from pydub import AudioSegment

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

# Load the model

While submitting notebooks, you need to have internet disabled. So you can't download models from huggingface directly. One workaround is to download the models from huggingface, upload them to kaggle as datasets and then use them!

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = "/kaggle/input/bangla-speech-processing-banglaasr/BanglaASR"
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_path)
tokenizer = WhisperTokenizer.from_pretrained(model_path)
processor = WhisperProcessor.from_pretrained(model_path)
model = WhisperForConditionalGeneration.from_pretrained(model_path).to(device)

# Demo inference

We'll write an inference function where we'll give the path as input and it will return the transcription!

First we'll need to convert the audio sampling rate to 16k since WhisperFeatureExtractor requires audios to be sampled at 16k.

In [4]:
#This code is modified from the source code provided here https://huggingface.co/bangla-speech-processing/BanglaASR
def inference_fn(path):
    speech_array, sampling_rate = sf.read(mp3_path)
    speech_array = librosa.resample(np.asarray(speech_array), orig_sr=sampling_rate, target_sr=16000)
    input_features = feature_extractor(speech_array, sampling_rate=16000, return_tensors="pt").input_features
    predicted_ids = model.generate(inputs=input_features.to(device))[0]
    transcription = processor.decode(predicted_ids, skip_special_tokens=True)
    return transcription


# Inferences

In [6]:
mp3_path = "/kaggle/input/kishorehanj-2/rec_26_audio_16.wav"
print(f"File name :",mp3_path)
AudioSegment.from_file(mp3_path)

File name : /kaggle/input/kishorehanj-2/rec_26_audio_16.wav


All set. Let's hit the "submit" option and pray!

In [12]:
import os
test_path = "/kaggle/input/kishoreganj-1"
files = os.listdir(test_path)
ids = []
sentences = []
for file in tqdm(files):
    ids.append(file.split(".")[0])
    mp3_path = os.path.join(test_path,file)
    prediction = inference_fn(mp3_path)
    
    #sanity check
    if len(prediction)==0:
        prediction = "<>"
    
    sentences.append(prediction)
    
    

  0%|          | 0/1000 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 1000/1000 [4:30:03<00:00, 16.20s/it] 


In [13]:
df1 = pd.DataFrame({"External_ID":ids,"Contents":sentences})
df1.head()

,External_ID,Contents
0,rec_04_audio_44,"কোচে- সাসি বাসেও- সাসি লাগে, হে- আজাও অন্কিয় ..."
1,rec_19_audio_24,"- হয়, আয় কবি- তিনি আর সাপান শেষ্ট এড়ালাই নু-..."
2,rec_05_audio_0,আসলাম আলীকুম।ারা জাম্বালয়ে কীশোয়গঞ্জি জায়া ...
3,rec_05_audio_18,পরেকেছেও এ প্রতিদিন জমির নান্ত হয়ে কে মাসা না...
4,rec_18_audio_10,বইয়ের কমিয়ের কমিয়ের কমিযের কমিযের কমিযের কম...


In [14]:
df1

,External_ID,Contents
0,rec_04_audio_44,"কোচে- সাসি বাসেও- সাসি লাগে, হে- আজাও অন্কিয় ..."
1,rec_19_audio_24,"- হয়, আয় কবি- তিনি আর সাপান শেষ্ট এড়ালাই নু-..."
2,rec_05_audio_0,আসলাম আলীকুম।ারা জাম্বালয়ে কীশোয়গঞ্জি জায়া ...
3,rec_05_audio_18,পরেকেছেও এ প্রতিদিন জমির নান্ত হয়ে কে মাসা না...
4,rec_18_audio_10,বইয়ের কমিয়ের কমিয়ের কমিযের কমিযের কমিযের কম...
...,...,...
995,rec_23_audio_35,খসও ও কমিতানে। হলুইব। আলো কীন বাইন? মাত!
996,rec_09_audio_19,"বিভার্তবকর্গ, ব্যাং- ফেলাগুড়া, ফেলাগুড়ার ব্য..."
997,rec_04_audio_6,সহিদ স্নাকাডো!
998,rec_16_audio_7,যেহজন্যা হয়েই যাম্যবায় ইভূন্যা হয়ে।


In [15]:
df1.to_csv("submission1.csv",index=False)

In [ ]:
=========================================================================================================================================

In [5]:
import os
test_path = "/kaggle/input/kishorehanj-2"
files = os.listdir(test_path)
ids = []
sentences = []
for file in tqdm(files):
    ids.append(file.split(".")[0])
    mp3_path = os.path.join(test_path,file)
    prediction = inference_fn(mp3_path)
    
    #sanity check
    if len(prediction)==0:
        prediction = "<>"
    
    sentences.append(prediction)
    
    

  0%|          | 0/1000 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 1000/1000 [4:24:38<00:00, 15.88s/it] 


In [6]:
df2 = pd.DataFrame({"External_ID":ids,"Contents":sentences})
df2.head()

,External_ID,Contents
0,rec_32_audio_29,"নিয়ত্তাক্লের আগে বধত।-বধতে ময়, এটি আকতার-আমি ..."
1,rec_43_audio_21,বরোবরে মিল ফেটটি বে কারকা নেমে গেলে তুমি দাদ্য...
2,rec_26_audio_43,"সুনামকে সুনাহাজি, তাই কিছু দ্বীয়, হাসিতে কোণি..."
3,rec_42_audio_4,গরু বাসউকিন্তানিয়া গরুব মাটার তাঁনটে আবজাতে আ...
4,rec_35_audio_36,দুই চেন্স হয়। দল তোকনো ছিল উলিশায় উলিশ বাসাইল...


In [7]:
df2

,External_ID,Contents
0,rec_32_audio_29,"নিয়ত্তাক্লের আগে বধত।-বধতে ময়, এটি আকতার-আমি ..."
1,rec_43_audio_21,বরোবরে মিল ফেটটি বে কারকা নেমে গেলে তুমি দাদ্য...
2,rec_26_audio_43,"সুনামকে সুনাহাজি, তাই কিছু দ্বীয়, হাসিতে কোণি..."
3,rec_42_audio_4,গরু বাসউকিন্তানিয়া গরুব মাটার তাঁনটে আবজাতে আ...
4,rec_35_audio_36,দুই চেন্স হয়। দল তোকনো ছিল উলিশায় উলিশ বাসাইল...
...,...,...
995,rec_43_small_audio_0,এই যোগ
996,rec_29_audio_10,সারিজো না!
997,rec_45_audio_11,"কান্দা, টিনডা, টিনড়া করা হিসে, আকি নিসে, আয়ও..."
998,rec_26_audio_51,"চল,চল,চল,কাজীন জুল, চল,চল,চল,কালোকগণে বাজেমা দ..."


In [ ]:
df2.to_csv("submission2.csv",index=False)

In [ ]:
==================================================================================================================================================

In [8]:
import os
test_path = "/kaggle/input/kishoreganj-3"
files = os.listdir(test_path)
ids = []
sentences = []
for file in tqdm(files):
    ids.append(file.split(".")[0])
    mp3_path = os.path.join(test_path,file)
    prediction = inference_fn(mp3_path)
    
    #sanity check
    if len(prediction)==0:
        prediction = "<>"
    
    sentences.append(prediction)
    
    

  0%|          | 0/85 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 85/85 [19:55<00:00, 14.07s/it]


In [9]:
df3 = pd.DataFrame({"External_ID":ids,"Contents":sentences})
df3.head()

,External_ID,Contents
0,rec_47_audio_67,"সেয়ার সিংহ হুসরাসি। সেয়ারও আগ হুসরাসে, পলার ..."
1,rec_47_audio_49,এলাকা জ্যান মিলা জেলিন।
2,rec_46_audio_36,"বিমানুটি আছে, বিমানুটি বেলান লিভু, সুসুসু বিল..."
3,rec_47_audio_28,"আন্যাদা দিবু? হয়া দিবু, একটি তাকাল আলবহ, তাকা..."
4,rec_47_audio_34,"যেমার রিয়াত গেছে, আর সোটন আর সুমন গেছে।"


In [10]:
df3

,External_ID,Contents
0,rec_47_audio_67,"সেয়ার সিংহ হুসরাসি। সেয়ারও আগ হুসরাসে, পলার ..."
1,rec_47_audio_49,এলাকা জ্যান মিলা জেলিন।
2,rec_46_audio_36,"বিমানুটি আছে, বিমানুটি বেলান লিভু, সুসুসু বিল..."
3,rec_47_audio_28,"আন্যাদা দিবু? হয়া দিবু, একটি তাকাল আলবহ, তাকা..."
4,rec_47_audio_34,"যেমার রিয়াত গেছে, আর সোটন আর সুমন গেছে।"
...,...,...
80,rec_47_audio_7,দ্রহ্তায়পকা।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।...
81,rec_47_audio_30,"একজন ব্যাউটে, কমিলালয় বালেপায়ে, কমিলালয়ে, ক..."
82,rec_46_audio_38,আমুয়াটা নম্বয়ল্য জানা বা ইয়ারটা ইয়াবাজারে ...
83,rec_47_audio_0,ট্যানজেস-রয়েশে-


In [11]:
df3.to_csv("submission3.csv",index=False)

In [ ]:
==================================================================================================================================================

In [16]:
values1 = df1[['External_ID','Contents']]
values2 = df2[['External_ID','Contents']]
values3 = df3[['External_ID','Contents']]


dataframes = [values1, values2, values3]

join = pd.concat(dataframes)
join

,External_ID,Contents
0,rec_04_audio_44,"কোচে- সাসি বাসেও- সাসি লাগে, হে- আজাও অন্কিয় ..."
1,rec_19_audio_24,"- হয়, আয় কবি- তিনি আর সাপান শেষ্ট এড়ালাই নু-..."
2,rec_05_audio_0,আসলাম আলীকুম।ারা জাম্বালয়ে কীশোয়গঞ্জি জায়া ...
3,rec_05_audio_18,পরেকেছেও এ প্রতিদিন জমির নান্ত হয়ে কে মাসা না...
4,rec_18_audio_10,বইয়ের কমিয়ের কমিয়ের কমিযের কমিযের কমিযের কম...
...,...,...
80,rec_47_audio_7,দ্রহ্তায়পকা।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।...
81,rec_47_audio_30,"একজন ব্যাউটে, কমিলালয় বালেপায়ে, কমিলালয়ে, ক..."
82,rec_46_audio_38,আমুয়াটা নম্বয়ল্য জানা বা ইয়ারটা ইয়াবাজারে ...
83,rec_47_audio_0,ট্যানজেস-রয়েশে-


In [17]:
join.to_csv("3.2-kishoreganj-banglaWhisper-inferences.csv")

In [18]:
print('done')

done
